### Custom Image Processing
In this script, we develop and deploy the Scene class - a custom Python Class for handling landsat imagery as downloaded from the USGS Earth Explorer website: https://earthexplorer.usgs.gov/. 

The coding here expects images organized in the USGS fashion, with unchanged file extensions and folder pathing. Refer to the D:\Landscan_Padma_bridge folder on C.Fox's Aurora for an example of how to organize and arrange these folders. 

Library import - usual suspects, plus PIL (image processing in python) and matplotlib

In [1]:
from __future__ import division, absolute_import
import os, sys
import rasterio as rt
import geopandas as gpd
import pandas as pd
import rasterio.mask
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import logging
from rasterio.transform import guard_transform
import imageio
logger = logging.getLogger(__name__)

Setting file paths, and importing the AOI of the area of interest as a shapefile in WGS 1984

In [2]:
outpath = r'C:\Users\charl\Documents\GOST\Bangladesh'
match_pth = r'C:\Users\charl\Documents\GOST\Bangladesh\LC081370442013041201T1-SC20190207220137'
pth = r'C:\Users\charl\Documents\GOST\Bangladesh\LC081370442013070101T2-SC20190207220038'
AOI = gpd.read_file(os.path.join(outpath, r'Padma_1_4326.shp'))
AOI = AOI.to_crs({'init':'epsg:32646'})
AOI = AOI.geometry.iloc[0]

Defining the class object. The 'Scene' class is the basis for the entire processing script. Within it it contains a series of functions that edit the underlying object when called in sequence. 

NOTE - not all functions defined in the Scene class are called during processing, as the development of the processing sequence was iterative and took many directions. Several of these functions will be useful in future applications, so they are retained for reference only.

In [1]:
class Scene(object):
    
    
    def __init__(self, folder_path, AOI = None):
        
        # Instantiate a new Scene object for a given folder path. 
        #The folder path is then assigned as the '.path' attribute of the scene object
        
        self.path = folder_path
        if AOI != None:
            self.AOI = AOI
            
    def bytescaling(self, data, cmin=None, cmax=None, high=255, low=0):
        """
        This function converts the input image to uint8 dtype and scales accordingly. 
        The value range is set by the low and high parameters. (default 0-255). 
        If the input image already has dtype uint8, no scaling is done.
        :param data: 16-bit image data array
        :param cmin: bias scaling of small values (def: data.min())
        :param cmax: bias scaling of large values (def: data.max())
        :param high: scale max value to high. (def: 255)
        :param low: scale min value to low. (def: 0)
        :return: 8-bit image data array
        """
        if data.dtype == np.uint8:
            return data

        if high > 255:
            high = 255
        if low < 0:
            low = 0
        if high < low:
            raise ValueError("'high' should be greater than or equal to 'low'.")

        if cmin is None:
            cmin = data.min()
        if cmax is None:
            cmax = data.max()

        cscale = cmax - cmin
        if cscale == 0:
            cscale = 1

        scale = float(high - low) / cscale
        bytedata = (data - cmin) * scale + low
        return (bytedata.clip(low, high) + 0.5).astype(np.uint8)
    
    def parse_metadata(self):
        
        # Each landsat scene comes with a metadata file. 
        # This function parses the metadata file into dictionaries
        # load the metadata textfile
        
        files = list(os.walk(self.path))[0][2]
        for i in files:
            if i[-7:] == 'MTL.txt':
                tf = i
                self.base = i[:-8]
            
        l = open(os.path.join(self.path, tf), 'r')
        text = l.readlines()

        # we set up some empty dictionaries...
        self.mult_dict = {}
        self.add_dict = {}
        
        #... and then add to these dictionaries the following useful items:
        for tline in text:
            bag = tline.split(' ')
            
            # Reflectance adjustments
            if 'REFLECTANCE_MULT_BAND' in tline:
                self.mult_dict.update({bag[4]:float(bag[6][:-1])})
            elif 'REFLECTANCE_ADD_BAND' in tline:
                self.add_dict.update({bag[4]:float(bag[6][:-1])})
            
            # sun elevation
            elif 'SUN_ELEVATION' in tline:
                self.elevation = float(bag[6][:-1])
            
            # Acquisition date
            elif 'DATE_ACQUIRED' in tline:
                date_raw = str(bag[6][:-1])
                self.date_datetime = pd.to_datetime(date_raw)
                self.date = '%s %s %s' % (self.date_datetime.day, 
                                          self.date_datetime.month_name(), 
                                          self.date_datetime.year)
                
    def corrected_reflectance(self, arr, band_no, msk, elevation):
        
        # Based on the documentation provided by Landsat, here we perform a reflectance adjustment 
        # using the multiplication and addition parameters extracted from the image metadata file
        
        multi = self.mult_dict['REFLECTANCE_MULT_BAND_%s' % band_no] 
        add = self.add_dict['REFLECTANCE_ADD_BAND_%s' % band_no] 
        adj = (arr * multi) + add
        adj = adj / np.sin(np.deg2rad(self.elevation))
        adj = np.ma.masked_array(adj, msk)
        return adj
    
    def load_band(self, band_number, band_name, byte_scale = False, max_perc = 98):
        
        # in this essential function, we load the band number from the tif, with options to byte scale.
        # we pass in a band number, as well as band name. 
        # max_perc is passed to self.bytescaling as an input - and curtails the values of the band at that percentile.
        
        band_loc = self.base+'_sr_band%s.tif' % band_number
        band = rt.open(os.path.join(self.path, band_loc))
        
        # If an AOI is present, crop the array to this AOI
        if 'AOI' in self.__dict__.keys():
            band_arr_AOI_msk = rt.mask.mask(band, [self.AOI], all_touched = False, crop = True)
            band_arr = band_arr_AOI_msk[0][0]
            
            # set the mask object as a new property, self.AOI_mask
            msk = rt.mask.raster_geometry_mask(band, [self.AOI], all_touched=False, crop = True)
            self.AOI_mask = msk[0]
            
            if 'template_transform' not in self.__dict__.keys():
                self.template_transform = band_arr_AOI_msk[1]
        else:
            band_arr = band.read()[0]

        if byte_scale == True:
            # perform byte scaling if requied
            band_arr = self.bytescaling(band_arr, cmin = 0, cmax = np.percentile(band_arr, max_perc))
            
        setattr(self, band_name, band_arr)
        
        # if a cloud mask has already been imported, mask the array to non-cloudy areas and adjust band name accordingly
        if 'cloud_mask' in self.__dict__.keys():
        
            band_arr_cloud_msk = np.ma.masked_array(band_arr, self.cloud_mask[0])

            setattr(self, band_name+'_cloud_masked', band_arr_cloud_msk)
        
        # if an AOI mask is already present, mask the array outside of the AOI and adjust the band name accordingly
        if 'AOI_mask' in self.__dict__.keys():
        
            band_arr_AOI_msk = np.ma.masked_array(band_arr, self.AOI_mask)

            setattr(self, band_name+'_AOI_masked', band_arr_AOI_msk)
        
        # If this is the first band to be loaded, we define a template object. 
        # This is used later when sending processed information to file. 
        if 'template_array' not in self.__dict__.keys():
            self.template_array = band_arr
        if 'template_raster' not in self.__dict__.keys():
            self.template_raster = band
        if 'template_transform' not in self.__dict__.keys():
            self.template_transform = band.transform
    
    def generate_RGB_raster(self, red_band, green_band, blue_band, out_name):
        
        # this function allows the user to generate a tri-band raster from any loaded bands of the Class scene. 
        # in this way, we can write a processed scene to file so it can be easily visualized. 
        
        out_fn = os.path.join(self.path, '%s.tif' % out_name)

        # Update metadata. We copy the template raster's metadata to do this (defined by the first loaded band.)
        meta = self.template_raster.meta.copy()
        
        meta.update(compress='lzw', 
                    count = 3, 
                    transform = self.template_transform, 
                    width = self.template_array.shape[1], 
                    height = self.template_array.shape[0]
                    )

        with rt.open(out_fn, 'w', **meta) as out:

            out.write_band(1, red_band)
            out.write_band(2, green_band)
            out.write_band(3, blue_band)
    
    def calculate_NDVI(self, N, R, propery_name):
        
        # Simple function for generating an NDVI band from two passed in bands. 
        # NDVI set as a new property of the Class object, with the property name set by the passed variable.
        
        N = N.astype(int)
        R = R.astype(int)
        in_NDVI_array = ((N - R) / (N + R))
        NDVI_array_masked = np.ma.masked_array(in_NDVI_array, self.AOI_mask, fill_value = -0)
        NDVI_array_masked.data[NDVI_array_masked.mask == True] = 0
        setattr(self, propery_name, NDVI_array_masked)
        
    def generate_uniband_raster(self, band, out_name):
        
        # simple function for sending a single band to file. Used when generating an NDVI-only TIF for example. 
        
        out_fn = os.path.join(self.path, '%s.tif' % out_name)
        
        meta = self.template_raster.meta.copy()
        
        meta.update(dtype = rasterio.float64, 
                    compress='lzw', 
                    count = 1, 
                    transform = self.template_transform, 
                    width = self.template_array.shape[1], 
                    height = self.template_array.shape[0],
                    nodata = band[0].data[0]
                    )
                
        with rt.open(out_fn, 'w', **meta) as out:

            out.write_band(1, band)
            
    def generate_uniband_image(self, band, property_name = 'NDVI_image', cmap = 'viridis'):
        
        # Instead of generating a TIF, this function generates a PIL Image object from a single band. 
        # Really useful for quickly visualizing an image in-notebook.  
        # for valid strings for the colormap input variable (cmap), check out:
        # https://matplotlib.org/examples/color/colormaps_reference.html
        
        cm_hot = mpl.cm.get_cmap(cmap)
        T = np.ma.masked_array(band.data, band.mask)
        Ta = cm_hot(T.data)
        im = np.uint8(Ta * 255)
        f_im = Image.fromarray(im)
        self.im = Ta
        setattr(self, property_name, f_im)
        
    def generate_triband_image(self, red, green, blue, property_name = 'RGB_image', max_perc = 98):
        
        # similar to the previous function, this function generates an RGB triband image from passed red, green and blue arrays
        
        rgbArray = np.zeros((red.shape[0],red.shape[1],3), 'uint8')
        rgbArray[..., 0] = red
        rgbArray[..., 1] = green
        rgbArray[..., 2] = blue
        f_im = Image.fromarray(rgbArray)
        setattr(self, property_name, f_im)
    
    def add_labels(self, img):
        
        # this custom function edits an image object to add a label in the bottom right corner. 
        # edit the message variable to change the contents of this label. 
        
        height = 60
        vertical_buffer = int(height / 4)
        text_indent = int(height / 4)
        text_size = int(height - vertical_buffer) 
        width = img.size[0] + 1
        message = "GOST 2019  |  Image Date: %s" % self.date

        img = img.crop((0,0,img.size[0],img.size[1]+height))

        draw = ImageDraw.Draw(img)

        font = ImageFont.truetype(os.path.join(os.getcwd(),'calibri', "Calibri.ttf"), text_size)

        draw.polygon([(0, img.size[1]), (0, img.size[1]-height), (width, img.size[1]-height), (width, img.size[1])], 
                     fill=(50,50,50), 
                     outline=None
                    )

        draw.text((text_indent,((img.size[1] - (text_size + (height - text_size)/2)))),
                  message,
                  (255,255,255),
                  font=font
                 )

        return img
    
    def import_qa_mask(self):
        
        # landsat scenes come bundled with a quality assurance (qa) mask. 
        # this mask identifies pixels which are cloudy. These have a special code. 
        # we can use this when scaling / calculating index values like NDVI - stripping these pixels out improves scores.
        # these files have the _pixel_qa.tif extension with them. 
        
        qa_loc = os.path.join(self.path, self.base+'_pixel_qa.tif')
        qa_raster = rt.open(qa_loc, 'r+')
        
        if 'AOI' in self.__dict__.keys():
            qa_msk = rt.mask.raster_geometry_mask(qa_raster, [self.AOI], all_touched = False, crop = True)[0]
            qa_data = rt.mask.mask(qa_raster, [self.AOI], all_touched = False, crop = True)[0]
            if 'AOI_mask' not in self.__dict__.keys():
                self.AOI_mask = qa_msk
        else:
            qa_msk = rt.read(qa_raster)
        
        # here we mask out permanent water features
        self.qa_mask = np.ma.masked_outside(qa_data,321,323).mask
        
        # here we mask out clouds
        self.cloud_mask = np.ma.masked_outside(qa_data,321,325).mask

        # and here we define the cloud cover percentage for the scene and set as an attribute. 
        self.cloud_cover = (1 - (self.cloud_mask == False).sum() / (self.AOI_mask == False).sum()) * 100
        
    def match_band(self, source, reference, property_name):
        
        # function for matching the histogram of one band to another. Calls the self.histogram_match function
        
        i = source.astype(np.int16)
        r = reference.astype(np.int16)
        
        matched_i = self.histogram_match(i, r)
        matched_i.data[matched_i.mask == True] = 0
        
        setattr(self, property_name,matched_i)
    
    def histogram_match(self, source, reference, match_proportion=1.0):
        """
        Adjust the values of a source array
        so that its histogram matches that of a reference array
        Parameters:
        -----------
            source: np.ndarray
            reference: np.ndarray
            match_proportion: float, range 0..1
        Returns:
        -----------
            target: np.ndarray
                The output array with the same shape as source
                but adjusted so that its histogram matches the reference
        """
        orig_shape = source.shape
        source = source.ravel()

        if np.ma.is_masked(reference):
            logger.debug("ref is masked, compressing")
            reference = reference.compressed()
        else:
            logger.debug("ref is unmasked, raveling")
            reference = reference.ravel()

        # get the set of unique pixel values
        # and their corresponding indices and counts
        logger.debug("Get unique pixel values")
        s_values, s_idx, s_counts = np.unique(
            source, return_inverse=True, return_counts=True)
        r_values, r_counts = np.unique(reference, return_counts=True)
        s_size = source.size

        if np.ma.is_masked(source):
            logger.debug("source is masked; get mask_index and remove masked values")
            mask_index = np.ma.where(s_values.mask)
            inter = np.ma.where(s_idx != mask_index[0])
            s_size = inter[0].size
            s_values = s_values.compressed()
            s_counts = np.delete(s_counts, mask_index)

        # take the cumsum of the counts; empirical cumulative distribution
        logger.debug("calculate cumulative distribution")
        s_quantiles = np.cumsum(s_counts) / s_size
        r_quantiles = np.cumsum(r_counts) / reference.size

        # find values in the reference corresponding to the quantiles in the source
        logger.debug("interpolate values from source to reference by cdf")
        interp_r_values = np.interp(s_quantiles, r_quantiles, r_values)

        if np.ma.is_masked(source):
            logger.debug("source is masked, add fill_value back at mask_index")
            interp_r_values = np.insert(interp_r_values, mask_index[0], source.fill_value)

        # using the inverted source indicies, pull out the interpolated pixel values
        logger.debug("create target array from interpolated values by index")
        target = interp_r_values[s_idx]

        # interpolation b/t target and source
        # 1.0 = full histogram match
        # 0.0 = no change
        if match_proportion is not None and match_proportion != 1:
            diff = source - target
            target = source - (diff * match_proportion)

        if np.ma.is_masked(source):
            logger.debug("source is masked, remask those pixels by position index")
            target = np.ma.masked_where(s_idx == mask_index[0], target)
            target.fill_value = source.fill_value

        return target.reshape(orig_shape)

Having defined the Class object and its associated functions, we define a function which calls the sub-functions in a pre-defined sequence. Most functions are appropriately named to make comprehension easy.

In [24]:
def ProcessImage(pth, AOI, R, completed_dates, max_cloud, matching = True, max_perc = 98):
    
    # Instantiate scene object
    Y = Scene(pth, AOI) 
    
    # extract metadata, assign to attributes of class
    Y.parse_metadata() 
    
    # Import quality assurance tif, define cloud cover %
    Y.import_qa_mask() 
    print('Image date: %s, Cloud cover: %s percent' % (Y.date, Y.cloud_cover))

    # Only process images which are below the max_cloud threshold
    if Y.cloud_cover <= max_cloud:
        print('processing...')
        
        # load bands with bytescaling, unless otherwise mentioned. 
        # bytes not scaled for NDVI - the red and NIR bands are loaded twice accordingly, but named differently. 
        Y.load_band(4, 'red', byte_scale = True, max_perc = max_perc)
        Y.load_band(4, 'red_unfixed', byte_scale = False)
        Y.load_band(3, 'green', byte_scale = True, max_perc = max_perc)
        Y.load_band(2, 'blue', byte_scale = True, max_perc = max_perc)
        Y.load_band(5, 'NIR', byte_scale = True, max_perc = max_perc)
        Y.load_band(5, 'NIR_unfixed', byte_scale = False)

        if matching == True:
            
            # Histogram Match if desired
            Y.match_band(Y.red_AOI_masked, R.red_cloud_masked, 'red_AOI_masked')
            Y.match_band(Y.blue_AOI_masked, R.blue_cloud_masked, 'blue_AOI_masked')
            Y.match_band(Y.green_AOI_masked, R.green_cloud_masked, 'green_AOI_masked')
            Y.match_band(Y.NIR_AOI_masked, R.NIR_cloud_masked, 'NIR_AOI_masked')
            Y.match_band(Y.NIR_unfixed_AOI_masked, R.NIR_unfixed_cloud_masked, 'NIR_unfixed_AOI_masked')
            Y.match_band(Y.red_unfixed_AOI_masked, R.red_unfixed_cloud_masked, 'red_unfixed_AOI_masked')

        # Generate NDVI
        Y.calculate_NDVI(Y.NIR_unfixed_AOI_masked, Y.red_unfixed_AOI_masked, 'NDVI')
        
        # Generate NDVI image
        Y.generate_uniband_image(Y.NDVI, 'NDVI_image')
        
        # Add labels to NDVI image
        Y.NDVI_image = Y.add_labels(Y.NDVI_image)
        
        # Save down NDVI image
        Y.NDVI_image.save(os.path.join(outpath, 'output_images','NDVI','%s-%s-%s_NDVI.png' % (Y.date_datetime.strftime('%y'),
                                                                                              Y.date_datetime.strftime('%m'),
                                                                                              Y.date_datetime.strftime('%d'))))

        # Generate RGB image
        Y.generate_triband_image(Y.red_AOI_masked, Y.green_AOI_masked, Y.blue_AOI_masked, 'RGB')
        
        # Add labels
        Y.RGB = Y.add_labels(Y.RGB)
        
        # Save down RGB image
        Y.RGB.save(os.path.join(outpath, 'output_images','RGB','%s-%s-%s_RGB.jpeg' % (Y.date_datetime.strftime('%y'),
                                                                                              Y.date_datetime.strftime('%m'),
                                                                                              Y.date_datetime.strftime('%d'))))

        # Generate a False Color image from NIR, Red and Blue
        Y.generate_triband_image(Y.NIR_AOI_masked, Y.red_AOI_masked, Y.blue_AOI_masked, 'false_color')
        
        # Add labels
        Y.false_color = Y.add_labels(Y.false_color)
        
        # Save down image
        Y.false_color.save(os.path.join(outpath, 'output_images','FC','%s-%s-%s_false_color.jpeg' % (Y.date_datetime.strftime('%y'),
                                                                                              Y.date_datetime.strftime('%m'),
        
        # Save the date object as a property for keeping track of processed images                                                                                      Y.date_datetime.strftime('%d'))))
        d_obj = Y.date_datetime
        
        return Y

    else:
        print('cloud cover fraction too high - passing over image')
    return Y

### Process Imagery

Here we iterate through the imagery folder, calling the Process Image function on each image. In each case, beforehand, we load R - the reference image - which is passed to the ProcessImage function for the purposes of band-matching each image to this reference image. This ensures consistency across bands and time.

In [25]:
dates = []
completed_dates = []

for root, folders, files in os.walk(r'D:\Landscan_Padma_bridge\imagery'):
    for folder in folders:
        max_perc = 98
        
        # load reference image, R, from the match_pth folder. 
        R = Scene(match_pth, AOI)
        R.parse_metadata()
        R.import_qa_mask()
        R.load_band(4, 'red', byte_scale = True, max_perc = max_perc)
        R.load_band(4, 'red_unfixed', byte_scale = False)
        R.load_band(3, 'green', byte_scale = True, max_perc = max_perc)
        R.load_band(2, 'blue', byte_scale = True, max_perc = max_perc)
        R.load_band(5, 'NIR', byte_scale = True, max_perc = max_perc)
        R.load_band(5, 'NIR_unfixed', byte_scale = False)
        
        # process the image in question
        p = os.path.join(root, folder)
        Y = ProcessImage(p, AOI, R, completed_dates, max_cloud = 5, matching = True, max_perc = max_perc)
        
        # record completed images
        completed_dates.append(Y.date)

Image date: 17 July 2013, Cloud cover: 99.10227847153314 percent
cloud cover fraction too high - passing over image
Image date: 2 August 2013, Cloud cover: 99.19364199726658 percent
cloud cover fraction too high - passing over image
Image date: 18 August 2013, Cloud cover: 100.0 percent
cloud cover fraction too high - passing over image
Image date: 19 September 2013, Cloud cover: 99.99400894913222 percent
cloud cover fraction too high - passing over image
Image date: 6 November 2013, Cloud cover: 93.7824678091674 percent
cloud cover fraction too high - passing over image
Image date: 8 December 2013, Cloud cover: 100.0 percent
cloud cover fraction too high - passing over image
Image date: 24 December 2013, Cloud cover: 93.01128928095935 percent
cloud cover fraction too high - passing over image
Image date: 25 January 2014, Cloud cover: 93.21336381180365 percent
cloud cover fraction too high - passing over image
Image date: 10 February 2014, Cloud cover: 93.55899269539395 percent
cloud c

Image date: 12 April 2013, Cloud cover: 4.039120984080324 percent
processing...
Image date: 28 April 2013, Cloud cover: 95.60184330435939 percent
cloud cover fraction too high - passing over image
Image date: 28 April 2013, Cloud cover: 95.60184330435939 percent
cloud cover fraction too high - passing over image
Image date: 14 May 2013, Cloud cover: 100.0 percent
cloud cover fraction too high - passing over image
Image date: 30 May 2013, Cloud cover: 60.99569944883033 percent
cloud cover fraction too high - passing over image
Image date: 15 June 2013, Cloud cover: 48.81014802068794 percent
cloud cover fraction too high - passing over image
Image date: 15 June 2013, Cloud cover: 48.81014802068794 percent
cloud cover fraction too high - passing over image
Image date: 1 July 2013, Cloud cover: 26.726436204388182 percent
cloud cover fraction too high - passing over image
Image date: 17 July 2013, Cloud cover: 63.12037358422754 percent
cloud cover fraction too high - passing over image
Imag

Image date: 20 May 2015, Cloud cover: 79.50025478365545 percent
cloud cover fraction too high - passing over image
Image date: 5 June 2015, Cloud cover: 43.467076289857644 percent
cloud cover fraction too high - passing over image
Image date: 5 June 2015, Cloud cover: 43.467076289857644 percent
cloud cover fraction too high - passing over image
Image date: 21 June 2015, Cloud cover: 99.30617610350862 percent
cloud cover fraction too high - passing over image
Image date: 21 June 2015, Cloud cover: 99.30617610350862 percent
cloud cover fraction too high - passing over image
Image date: 7 July 2015, Cloud cover: 100.0 percent
cloud cover fraction too high - passing over image
Image date: 7 July 2015, Cloud cover: 100.0 percent
cloud cover fraction too high - passing over image
Image date: 23 July 2015, Cloud cover: 98.94480179800948 percent
cloud cover fraction too high - passing over image
Image date: 23 July 2015, Cloud cover: 98.94480179800948 percent
cloud cover fraction too high - pa

Image date: 18 February 2017, Cloud cover: 16.606847895388544 percent
cloud cover fraction too high - passing over image
Image date: 18 February 2017, Cloud cover: 16.606847895388544 percent
cloud cover fraction too high - passing over image
Image date: 6 March 2017, Cloud cover: 16.164238221487004 percent
cloud cover fraction too high - passing over image
Image date: 22 March 2017, Cloud cover: 46.38477993831035 percent
cloud cover fraction too high - passing over image
Image date: 7 April 2017, Cloud cover: 56.58683445462758 percent
cloud cover fraction too high - passing over image
Image date: 23 April 2017, Cloud cover: 100.0 percent
cloud cover fraction too high - passing over image
Image date: 9 May 2017, Cloud cover: 100.0 percent
cloud cover fraction too high - passing over image
Image date: 25 May 2017, Cloud cover: 85.52558052517435 percent
cloud cover fraction too high - passing over image
Image date: 10 June 2017, Cloud cover: 49.75740164981648 percent
cloud cover fraction 

### Make GIFs

In order to quickly appreciate changes in imagery, we can generate some .gifs. This block processes the images in three folders - RGB, FC and NDVI - and generates .gifs of each. 

In [26]:
types = ['RGB','FC','NDVI']
for t in types:
    for root, folders, files in os.walk(r'C:\Users\charl\Documents\GOST\Bangladesh\output_images\%s' % t):
        pass
    duration = [0.75] * len(files)
    duration[0] = 5
    duration[-1] = 5
    
    with imageio.get_writer(os.path.join(outpath,'output_images','GIFs','%s.gif' % t), mode='I', duration = duration) as writer:
        for root, folders, files in os.walk(r'C:\Users\charl\Documents\GOST\Bangladesh\output_images\%s' % t):
            for f in files:
                print(f)
                image = imageio.imread(os.path.join(root,f))
                writer.append_data(image)

13-04-12_RGB.jpeg
13-11-06_RGB.jpeg
13-12-24_RGB.jpeg
14-01-25_RGB.jpeg
14-02-10_RGB.jpeg
14-02-26_RGB.jpeg
14-03-30_RGB.jpeg
14-11-25_RGB.jpeg
15-01-28_RGB.jpeg
15-02-13_RGB.jpeg
15-04-18_RGB.jpeg
15-09-25_RGB.jpeg
15-10-27_RGB.jpeg
15-11-12_RGB.jpeg
15-11-28_RGB.jpeg
15-12-30_RGB.jpeg
16-01-15_RGB.jpeg
16-02-16_RGB.jpeg
16-03-03_RGB.jpeg
16-11-14_RGB.jpeg
16-11-30_RGB.jpeg
16-12-16_RGB.jpeg
17-01-17_RGB.jpeg
17-02-02_RGB.jpeg
17-11-01_RGB.jpeg
17-12-03_RGB.jpeg
18-01-04_RGB.jpeg
18-01-20_RGB.jpeg
18-10-03_RGB.jpeg
18-11-20_RGB.jpeg
18-12-22_RGB.jpeg
13-04-12_false_color.jpeg
13-11-06_false_color.jpeg
13-12-24_false_color.jpeg
14-01-25_false_color.jpeg
14-02-10_false_color.jpeg
14-02-26_false_color.jpeg
14-03-30_false_color.jpeg
14-11-25_false_color.jpeg
15-01-28_false_color.jpeg
15-02-13_false_color.jpeg
15-04-18_false_color.jpeg
15-09-25_false_color.jpeg
15-10-27_false_color.jpeg
15-11-12_false_color.jpeg
15-11-28_false_color.jpeg
15-12-30_false_color.jpeg
16-01-15_false_color.jpeg
